In [1]:
import torch

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [4]:
path = '/cognitive_comp/lincong/models/UltraCM-13b'

In [40]:
tokenizer = AutoTokenizer.from_pretrained(path, use_fast=True)

In [6]:
model = AutoModelForCausalLM.from_pretrained(path)

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:36<00:00, 12.05s/it]
Some weights of LlamaForCausalLM were not initialized from the model checkpoint at /cognitive_comp/lincong/models/UltraCM-13b and are newly initialized: ['model.layers.25.self_attn.rotary_emb.inv_freq', 'model.layers.20.self_attn.rotary_emb.inv_freq', 'model.layers.27.self_attn.rotary_emb.inv_freq', 'model.layers.39.self_attn.rotary_emb.inv_freq', 'model.layers.28.self_attn.rotary_emb.inv_freq', 'model.layers.33.self_attn.rotary_emb.inv_freq', 'model.layers.24.self_attn.rotary_emb.inv_freq', 'model.layers.0.self_attn.rotary_emb.inv_freq', 'model.layers.36.self_attn.rotary_emb.inv_freq', 'model.layers.17.self_attn.rotary_emb.inv_freq', 'model.layers.9.self_attn.rotary_emb.inv_freq'

In [20]:
model = model.cuda()

In [12]:
tokenizer.pad_token_id = 0

In [13]:
system_prompt = "User: A one-turn chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, very detailed, and polite answers to the user's questions.</s>"

ultracm_instruction_template = """Given my answer to an instruction, your role is to provide specific and constructive feedback for me. You should find the best way for me to learn from your feedback and improve my performance. 

You should consider multiple aspects of my answer, including helpfulness, truthfulness, honesty, and to what extent the answer follows instructions.
---

### Instruction
{instruction}

### Answer
{completion}
---

Please act as a teacher and provide specific and constructive feedback. Besides describing the weaknesses of the answer, you should also provide specific suggestions to guide me toward understanding how to improve. Please note, however, that your suggestions should help me better complete the instructions, but you should not introduce new requirements that are not mentioned in the instructions. Your feedback should focus on enhancing my ability to think critically and respond accurately. However, never explicitly provide the reference answer, nor do polite phrases be required. Only respond with concise feedback in chat style. Finally, score the overall quality of the answer from 1 to 10, where 1 is the worst and 10 is the best.

*Format*
### Feedback
Overall Score: [1-10]
[Your feedback]

---

### Feedback
Overall Score: 
"""


def get_input_text(query, response):
    conv = [system_prompt]
    conv.append("User: " + ultracm_instruction_template.format(
                                    instruction=query,
                                    completion=response,
                                    ) + "</s>")
    conv.append("Assistant: ")
    prompt = "\n".join(conv)
    return prompt

In [41]:
q = "Where is it humid every single day?"
r = """There are several places around the world where it is humid every single day. Here are some examples:

1. Hawaii, USA: Known for its tropical climate, Hawaii is one of the most humid places on Earth, with an average relative humidity of 60-80% throughout the year.
2. Maldives: This island nation in the Indian Ocean has a tropical monsoon climate, with temperatures and humidity levels remaining high throughout the year.
3. Singapore: With an average relative humidity of 70-80%, Singapore is one of the most humid cities in the world. The city-state's tropical rainforest climate and proximity to the equator contribute to its high humidity levels.
4. Brunei: Located on the island of Borneo, Brunei has a tropical rainforest climate with high temperatures and humidity levels year-round.
5. Costa Rica: This Central American country has a tropical climate with high humidity levels, especially in the rainy season (May to November).
6. Panama: Panama's tropical climate and location near the equator result in high humidity levels throughout the year.
7. Darwin, Australia: Darwin experiences a tropical climate with high humidity levels, especially during the wet season (December to April).
8. Cairns, Australia: Cairns is a coastal city in tropical North Queensland, Australia, with high humidity levels throughout the year.
9. Puerto Rico: This Caribbean island has a tropical climate with high humidity levels, especially during the summer months.
10. Florida Keys, USA: The Florida Keys have a subtropical climate with high humidity levels, especially during the summer months.

Please note that these are just a few examples, and there may be other places around the world that experience high humidity levels as well."""

In [42]:
prompt = get_input_text(q, r)

In [43]:
print(prompt)

User: A one-turn chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, very detailed, and polite answers to the user's questions.</s>
User: Given my answer to an instruction, your role is to provide specific and constructive feedback for me. You should find the best way for me to learn from your feedback and improve my performance. 

You should consider multiple aspects of my answer, including helpfulness, truthfulness, honesty, and to what extent the answer follows instructions.
---

### Instruction
Where is it humid every single day?

### Answer
There are several places around the world where it is humid every single day. Here are some examples:

1. Hawaii, USA: Known for its tropical climate, Hawaii is one of the most humid places on Earth, with an average relative humidity of 60-80% throughout the year.
2. Maldives: This island nation in the Indian Ocean has a tropical monsoon climate, with temperatures and humidity levels remaining high

In [44]:
input_ids = tokenizer(prompt, return_tensors='pt').input_ids

In [37]:
gen_args = {'temperature': 1.0, 'top_p': 1.0, 'max_new_tokens': 1024, 'repetition_penalty': 1.2, 'do_sample': True, 'eos_token_id': tokenizer.eos_token_id, 'pad_token_id': tokenizer.pad_token_id}

In [38]:
output = model.generate(input_ids.cuda(), **gen_args)

In [39]:
print(tokenizer.batch_decode(output)[0])

<s> User: A one-turn chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, very detailed, and polite answers to the user's questions.</s> 
User: Given my answer to an instruction, your role is to provide specific and constructive feedback for me. You should find the best way for me to learn from your feedback and improve my performance. 

You should consider multiple aspects of my answer, including helpfulness, truthfulness, honesty, and to what extent the answer follows instructions.
---

### Instruction
Where is it humid every single day?

### Answer
There are several places around the world where it is humid every single day. Here are some examples:

1. Hawaii, USA: Known for its tropical climate, Hawaii is one of the most humid places on Earth, with an average relative humidity of 60-80% throughout the year.
2. Maldives: This island nation in the Indian Ocean has a tropical monsoon climate, with temperatures and humidity levels remaining

In [29]:
type(output)

torch.Tensor

In [45]:
int(' 9')

9

In [46]:
torch.randn(3).cuda()

tensor([-0.0479, -0.4691,  2.0417], device='cuda:0')

In [47]:
torch.flip?

Docstring:
flip(input, dims) -> Tensor

Reverse the order of an n-D tensor along given axis in dims.

.. note::
    `torch.flip` makes a copy of :attr:`input`'s data. This is different from NumPy's `np.flip`,
    which returns a view in constant time. Since copying a tensor's data is more work than viewing that data,
    `torch.flip` is expected to be slower than `np.flip`.

Args:
    input (Tensor): the input tensor.
    dims (a list or tuple): axis to flip on

Example::

    >>> x = torch.arange(8).view(2, 2, 2)
    >>> x
    tensor([[[ 0,  1],
             [ 2,  3]],

            [[ 4,  5],
             [ 6,  7]]])
    >>> torch.flip(x, [0, 1])
    tensor([[[ 6,  7],
             [ 4,  5]],

            [[ 2,  3],
             [ 0,  1]]])
Type:      builtin_function_or_method

In [48]:
torch.Tensor.flip?

Docstring:
flip(dims) -> Tensor

See :func:`torch.flip`
Type:      method_descriptor

In [50]:
a = torch.randn(3,3).flip(dims=[-1])

In [51]:
a

tensor([[-0.1242, -1.2809,  0.8646],
        [ 0.1631,  1.3011,  0.7660],
        [ 0.4409,  1.3260,  0.5626]])

In [52]:
a.flip(dims=[-1])

tensor([[ 0.8646, -1.2809, -0.1242],
        [ 0.7660,  1.3011,  0.1631],
        [ 0.5626,  1.3260,  0.4409]])